# Aufbau der Notebooks [Philipp]
- Multivalue bei Notebooks angucken 
- Wie installiere ich den "scheiß"

In [ ]:
# Globals
from enum import Enum

PATH_RAW_DATA = "./raw/"
PATH_FEATURE_DATA = "./features/"
PATH_EXPLORATION_DATA = "./exploration/"
PATH_MODEL = "./model/"
DATA_SOURCE = Enum("DATA_SOURCE", ["KIDAQ", "VIB"])
FEATURE = Enum(
    "FEATURE",
    [
        Enum(
            DATA_SOURCE.KIDAQ,
            [
                "RPM",
                "POWER",
                "P1_STD",
                "P2_STD",
                "P_DIFF_STD",
                "P1_RANGE",
                "P2_RANGE",
                "P_DIFF_RANGE",
                "P1_IQR",
                "P2_IQR",
                "P_DIFF_IQR",
                "P1_MEAN_MEDIAN",
                "P2_MEAN_MEDIAN",
                "P_DIFF_MEAN_MEDIA",
                "P1_FFT",
                "P2_FFT",
                "P_DIFF_FFT",
            ],
        ),
        Enum(
            DATA_SOURCE.VIB,
            [
                "RPM",
                "POWER",
                "S1_RANGE",
                "S2_RANGE",
                "S3_RANGE",
                "S4_RANGE",
                "S5_RANGE",
                "S6_RANGE",
                "S7_RANGE",
                "S8_RANGE",
                "S1_STD",
                "S2_STD",
                "S3_STD",
                "S4_STD",
                "S5_STD",
                "S6_STD",
                "S7_STD",
                "S8_STD",
                "S1_IQR",
                "S2_IQR",
                "S3_IQR",
                "S4_IQR",
                "S5_IQR",
                "S6_IQR",
                "S7_IQR",
                "S8_IQR",
                "S1_MEAN_MEDIAN",
                "S2_MEAN_MEDIAN",
                "S3_MEAN_MEDIAN",
                "S4_MEAN_MEDIAN",
                "S5_MEAN_MEDIAN",
                "S6_MEAN_MEDIAN",
                "S7_MEAN_MEDIAN",
                "S8_MEAN_MEDIAN",
                "S1_FFT",
                "S2_FFT",
                "S3_FFT",
                "S4_FFT",
                "S5_FFT",
                "S6_FFT",
                "S7_FFT",
                "S8_FFT",
            ],
        ),
    ],
)
OPERATING_POINT_FREQ = Enum(
    "OPERATING_POINT_RPM", ["RPM725", "RPM1450", "RPM2175", "RPM2900"]
)
OPERATING_POINT_FLOW = Enum(
    "OPERATING_POINT_FLOW_RATE", ["PCT0", "PCT25", "PCT50", "PCT75", "PCT100"]
)
LEARNER = Enum("LEARNER", ["DNN", "DT"])

DEFAULT_LEARNER = LEARNER.DNN
DEFAULT_RAW_DATA = FEATURE.DATA_SOURCE.KIDAQ

## 1. Aufgabe und Daten erklären/beschreiben [Philipp]

## 3. Preprocessing [Valerij]
- Auswahl der Fenstergröße in Millisekunden
- Multi-Selektion der Aggregation (avg, mean, std, ...)
- Frequenzanalyse
- Fourier-Transformation
- Fenstergröße nach Frequenzbereichen

### 4. Deskriptive/Explorative Datenanalyse [Philipp]
- Plots
- Beschreibung der Plots

### 5. Machine Learning [Kevin]
Multi-Selektion für:
- die unterschiedlichen Features
- Auswahl der Featuredateien (Testdaten)
- Auswahl des Learners
- Konfiguration der Hyperparameter
- Live-Validation des Models mit vorausgewählten Testdaten (Random-Search, ...)

In [ ]:
FEATURE_TRAIN_FILE = "train.csv"
FEATURE_TEST_FILE = "test.csv"

DEFAULT_OPERATING_POINTS = [
    OPERATING_POINT_FREQ.RPM725,
    OPERATING_POINT_FREQ.RPM1450,
    OPERATING_POINT_FREQ.RPM2175,
    OPERATING_POINT_FREQ.RPM2900,
    OPERATING_POINT_FLOW.PCT0,
    OPERATING_POINT_FLOW.PCT25,
    OPERATING_POINT_FLOW.PCT50,
    OPERATING_POINT_FLOW.PCT75,
    OPERATING_POINT_FLOW.PCT100
]


# DNN
DNN_EXPLORATION_TARGET_VAL_ACCURACY = 0.9
DNN_EXPLORATION_MAX_ITER = 100
DNN_EXPLORATION_HIDDEN_LAYERS_MIN = 2
DNN_EXPLORATION_HIDDEN_LAYERS_MAX = 4
DNN_EXPLORATION_HIDDEN_UNITS_MIN = 8
DNN_EXPLORATION_HIDDEN_UNITS_MAX = 64


DNN_EARLY_STOPPING_PATIENCE = 50
DNN_VERBOSE = 0
DNN_EPOCHS = 100
DNN_BATCH_SIZE = 32
DNN_BATCH_NORMALIZATION = True


# DT
DT_MAX_DEPTH = 6
DT_NUM_ESTIMATORS = 100

### 5. Modelanalyse des Learners [Kevin]
- Vorherige Auswahl eines Learners
- Feature Importance
- Korrelationsmatrix
- Konfusionsmatrix
- Post-Validation des Models mit auswählbaren Daten

### 6. Statische Interpretation des Resultats
- Welches Ergebnis haben wir erzielt und wie kann man es anwenden?